In [ ]:
with np.load('mnist.npz', allow_pickle=True) as f:
    X_train, y_train = f['x_train'], f['y_train']
    X_test, y_test = f['x_test'], f['y_test']
sample = 1
image = X_train[sample]
fig = plt.figure
plt.imshow(image, cmap='gray')
plt.show()
X_train = X_train.reshape(X_train.shape[0], 28 * 28)
X_test = X_test.reshape(X_test.shape[0], 28 * 28)



In [ ]:
n_estimators_range = range(1, 50, 1)
f_train_cb = []
f_test_cb = []
for n in n_estimators_range:
    cat_boost = CatBoostClassifier(n_estimators=n)
    cat_boost.fit(X_train, y_train)
    Y_predict_train = cat_boost.predict(X_train)
    Y_predict_test = cat_boost.predict(X_test)
    f_train_cb.append(f1_score(y_train, Y_predict_train, average='micro'))
    f_test_cb.append(f1_score(y_test, Y_predict_test, average='micro'))

plt.plot(n_estimators_range, f_train_cb)
plt.plot(n_estimators_range, f_test_cb)
plt.title("F мера для градиентного бустинга на основе MNIST")
plt.xlabel('n_estimators')
plt.ylabel('F-мера')
plt.legend(['f_train', 'f_test'])
plt.show()


def map(data):
    output = []
    for i in range(len(data)):
        if data[i][0] > data[i][1]:
            output.append(0)
        else:
            output.append(1)
    return output


model = CatBoostClassifier(n_estimators=5)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

y_pred = to_categorical(y_pred.reshape(-1), 10)
y_test_check = to_categorical(y_test, 10)

y_test_labels = np.argmax(y_test_check, axis=1)
y_pred_tr = np.argmax(y_pred, axis=1)

confusion_matrix_ = confusion_matrix(y_test_check, y_pred_tr)
cm = pd.DataFrame(data=confusion_matrix_, columns=['0', '1', '2', '3', '4',
                                                   '5', '6', '7', '8', '9'],
                  index=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'])
ax = sns.heatmap(cm, annot=True, fmt="d")


In [ ]:
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)
NB_CLASSES = y_train.shape[1]
INPUT_SHAPE = (X_train.shape[1],)
model = Sequential()
model.add(Input(shape=INPUT_SHAPE))
model.add(Dense(32))
model.add(Activation('relu'))
model.add(Dropout(0.3))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dense(8))
model.add(Activation('relu'))
model.add(Dense(NB_CLASSES))
model.add(Activation('softmax'))
model.summary()

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['Precision', 'Recall'])


In [ ]:
EPOCHS = 30
history = model.fit(X_train, y_train, batch_size=32, epochs=30, verbose=
1, validation_data=(X_test, y_test))
f1_score_list_train = []
f1_score_list_test = []
for i in range(EPOCHS):
    f1_score_list_train.append(2 * history.history['precision'][i] *
                               history.history['recall'][i] / (history.history['precision'][i] +
                                                               history.history['recall'][i]))
    f1_score_list_test.append(2 * history.history['val_precision'][i] *
                              history.history['val_recall'][i] / (history.history['val_precision'][i] +
                                                                  history.history['val_recall'][i]))